# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f43bce38e50>
├── 'a' --> tensor([[ 1.6248,  1.4398, -1.6301],
│                   [-1.4909,  0.9003, -0.1782]])
└── 'x' --> <FastTreeValue 0x7f43bce28730>
    └── 'c' --> tensor([[ 2.0149,  0.8415,  0.9243,  1.4636],
                        [ 0.8215, -1.8189, -0.4633,  0.9464],
                        [-0.4976, -0.8870, -0.9135, -1.2895]])

In [4]:
t.a

tensor([[ 1.6248,  1.4398, -1.6301],
        [-1.4909,  0.9003, -0.1782]])

In [5]:
%timeit t.a

60 ns ± 0.0562 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f43bce38e50>
├── 'a' --> tensor([[-0.1133,  1.1343, -0.1750],
│                   [ 0.1615,  0.0484,  0.3989]])
└── 'x' --> <FastTreeValue 0x7f43bce28730>
    └── 'c' --> tensor([[ 2.0149,  0.8415,  0.9243,  1.4636],
                        [ 0.8215, -1.8189, -0.4633,  0.9464],
                        [-0.4976, -0.8870, -0.9135, -1.2895]])

In [7]:
%timeit t.a = new_value

60.7 ns ± 0.029 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.6248,  1.4398, -1.6301],
               [-1.4909,  0.9003, -0.1782]]),
    x: Batch(
           c: tensor([[ 2.0149,  0.8415,  0.9243,  1.4636],
                      [ 0.8215, -1.8189, -0.4633,  0.9464],
                      [-0.4976, -0.8870, -0.9135, -1.2895]]),
       ),
)

In [10]:
b.a

tensor([[ 1.6248,  1.4398, -1.6301],
        [-1.4909,  0.9003, -0.1782]])

In [11]:
%timeit b.a

53.4 ns ± 0.09 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0767,  0.7945, -0.0631],
               [-1.0676, -0.8136,  0.3523]]),
    x: Batch(
           c: tensor([[ 2.0149,  0.8415,  0.9243,  1.4636],
                      [ 0.8215, -1.8189, -0.4633,  0.9464],
                      [-0.4976, -0.8870, -0.9135, -1.2895]]),
       ),
)

In [13]:
%timeit b.a = new_value

483 ns ± 0.0988 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

855 ns ± 0.216 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 62.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 392 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f43bce28070>
├── 'a' --> tensor([[[ 1.6248,  1.4398, -1.6301],
│                    [-1.4909,  0.9003, -0.1782]],
│           
│                   [[ 1.6248,  1.4398, -1.6301],
│                    [-1.4909,  0.9003, -0.1782]],
│           
│                   [[ 1.6248,  1.4398, -1.6301],
│                    [-1.4909,  0.9003, -0.1782]],
│           
│                   [[ 1.6248,  1.4398, -1.6301],
│                    [-1.4909,  0.9003, -0.1782]],
│           
│                   [[ 1.6248,  1.4398, -1.6301],
│                    [-1.4909,  0.9003, -0.1782]],
│           
│                   [[ 1.6248,  1.4398, -1.6301],
│                    [-1.4909,  0.9003, -0.1782]],
│           
│                   [[ 1.6248,  1.4398, -1.6301],
│                    [-1.4909,  0.9003, -0.1782]],
│           
│                   [[ 1.6248,  1.4398, -1.6301],
│                    [-1.4909,  0.9003, -0.1782]]])
└── 'x' --> <FastTreeValue 0x7f4314b39580>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 91.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f439831fca0>
├── 'a' --> tensor([[ 1.6248,  1.4398, -1.6301],
│                   [-1.4909,  0.9003, -0.1782],
│                   [ 1.6248,  1.4398, -1.6301],
│                   [-1.4909,  0.9003, -0.1782],
│                   [ 1.6248,  1.4398, -1.6301],
│                   [-1.4909,  0.9003, -0.1782],
│                   [ 1.6248,  1.4398, -1.6301],
│                   [-1.4909,  0.9003, -0.1782],
│                   [ 1.6248,  1.4398, -1.6301],
│                   [-1.4909,  0.9003, -0.1782],
│                   [ 1.6248,  1.4398, -1.6301],
│                   [-1.4909,  0.9003, -0.1782],
│                   [ 1.6248,  1.4398, -1.6301],
│                   [-1.4909,  0.9003, -0.1782],
│                   [ 1.6248,  1.4398, -1.6301],
│                   [-1.4909,  0.9003, -0.1782]])
└── 'x' --> <FastTreeValue 0x7f4314ab0580>
    └── 'c' --> tensor([[ 2.0149,  0.8415,  0.9243,  1.4636],
                        [ 0.8215, -1.8189, -0.4633,  0.9464],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 49 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.9 µs ± 63.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.6248,  1.4398, -1.6301],
                [-1.4909,  0.9003, -0.1782]],
       
               [[ 1.6248,  1.4398, -1.6301],
                [-1.4909,  0.9003, -0.1782]],
       
               [[ 1.6248,  1.4398, -1.6301],
                [-1.4909,  0.9003, -0.1782]],
       
               [[ 1.6248,  1.4398, -1.6301],
                [-1.4909,  0.9003, -0.1782]],
       
               [[ 1.6248,  1.4398, -1.6301],
                [-1.4909,  0.9003, -0.1782]],
       
               [[ 1.6248,  1.4398, -1.6301],
                [-1.4909,  0.9003, -0.1782]],
       
               [[ 1.6248,  1.4398, -1.6301],
                [-1.4909,  0.9003, -0.1782]],
       
               [[ 1.6248,  1.4398, -1.6301],
                [-1.4909,  0.9003, -0.1782]]]),
    x: Batch(
           c: tensor([[[ 2.0149,  0.8415,  0.9243,  1.4636],
                       [ 0.8215, -1.8189, -0.4633,  0.9464],
                       [-0.4976, -0.8870, -0.9135, -1.2895]],
         

In [26]:
%timeit Batch.stack(batches)

79 µs ± 218 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.6248,  1.4398, -1.6301],
               [-1.4909,  0.9003, -0.1782],
               [ 1.6248,  1.4398, -1.6301],
               [-1.4909,  0.9003, -0.1782],
               [ 1.6248,  1.4398, -1.6301],
               [-1.4909,  0.9003, -0.1782],
               [ 1.6248,  1.4398, -1.6301],
               [-1.4909,  0.9003, -0.1782],
               [ 1.6248,  1.4398, -1.6301],
               [-1.4909,  0.9003, -0.1782],
               [ 1.6248,  1.4398, -1.6301],
               [-1.4909,  0.9003, -0.1782],
               [ 1.6248,  1.4398, -1.6301],
               [-1.4909,  0.9003, -0.1782],
               [ 1.6248,  1.4398, -1.6301],
               [-1.4909,  0.9003, -0.1782]]),
    x: Batch(
           c: tensor([[ 2.0149,  0.8415,  0.9243,  1.4636],
                      [ 0.8215, -1.8189, -0.4633,  0.9464],
                      [-0.4976, -0.8870, -0.9135, -1.2895],
                      [ 2.0149,  0.8415,  0.9243,  1.4636],
                      [ 0.8215, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 412 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
